<a href="https://colab.research.google.com/github/Feny-rs/sprouts_model/blob/master/Inception_v3_imageNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstall

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="CLAcDuqXX6buLaKIgh29")
project = rf.workspace("politeknik-negeri-semarang-nurcn").project("yolo-sprouts-classification")
dataset = project.version(4).download("multiclass")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Yolo-Sprouts-Classification-4 in multiclass:: 100%|██████████| 1282/1282 [00:00<00:00, 3802.52it/s]


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your dataset directory
dataset_dir = "/content/Yolo-Sprouts-Classification-4"  # Replace with your dataset directory path

# Preprocessing and Augmentation Parameters
image_size = (299, 299)  # Adjust the size as needed
batch_size = 32
num_classes = 3  # Replace with your number of classes

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    zoom_range=(0.0, 0.7),
    brightness_range=(0.8, 1.2),
    fill_mode="constant",
    cval=255,
)

# Load the InceptionV3 model with pre-trained weights from ImageNet
base_model = keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(image_size[0], image_size[1], 3),
)

# Customize the top layers for your classification task
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(num_classes, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Prepare the dataset
train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",  # This is for multi-class classification
)

# Training the model
model.fit(train_data, epochs=10)  # You can adjust the number of epochs

# Save the model
model.save("custom_model.h5")

Found 1274 images belonging to 3 classes.
Epoch 1/10
40/40 [==============================] - 1279s 31s/step - loss: 0.6500 - accuracy: 0.8297
Epoch 2/10
40/40 [==============================] - 1243s 31s/step - loss: 0.5367 - accuracy: 0.8422
Epoch 3/10
40/40 [==============================] - 1255s 31s/step - loss: 0.5546 - accuracy: 0.8430
Epoch 4/10
40/40 [==============================] - 1250s 31s/step - loss: 0.5453 - accuracy: 0.8422
Epoch 5/10
40/40 [==============================] - 1257s 31s/step - loss: 0.5310 - accuracy: 0.8430
Epoch 6/10
40/40 [==============================] - 1215s 30s/step - loss: 0.5343 - accuracy: 0.8430
Epoch 7/10
40/40 [==============================] - 1197s 30s/step - loss: 0.5315 - accuracy: 0.8430
Epoch 8/10
40/40 [==============================] - 1223s 31s/step - loss: 0.5259 - accuracy: 0.8430
Epoch 9/10
40/40 [==============================] - 1180s 29s/step - loss: 0.5088 - accuracy: 0.8430
Epoch 10/10
40/40 [==============================

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
